# Ottomatic Reply Project

In [ ]:
!pip install langsmith mysql.connector

In [ ]:
# Import libraries
from google.cloud import language_v2
from langsmith import traceable
from google import genai
from google.genai import types
from google.genai.types import HttpOptions

In [ ]:
# Sample customer-service e-mails with tags
#Training Data

#[Product Consultation | Tone: Neutral | Urgency: Low]
email1 = """
Subject: Question about the Nebula-X Smart Projector before I buy

Hi team,

I’m Maria-Luisa Ortega and I’m really excited about the Nebula-X projector! A quick check before I place my order:

* Will it handle 220 V power in Spain?
* Is it compatible with my Samsung Galaxy S24 via USB-C?

If you need to reach me: mlortega1987@gmail.com, mobile +34 612 445 778.  
Shipping would go to Calle Mayor 12, 2-B, 28013 Madrid, Spain.

Thanks a lot—looking forward to your advice!

Best,  
María-Luisa
"""

#[Order Support | Tone: Unhappy | Urgency: Medium]
email2 = """

Subject: Trouble completing checkout – “Payment declined” error

Hello,

My name is Jonathan K. Barrett (phone +1 646-555-0194) and I’m trying to order Order ID draft #TEMP-789312 for the Tempo Fitness Watch.  
Your site keeps rejecting my Visa (•••• •••• •••• 4389 exp 11/27) and PayPal isn’t loading. Could someone walk me through a fix?  
I really want this before my hiking trip.

Billing address: 338 Hudson St, Apt 4C, New York, NY 10014, USA.

Thanks!

Jon
"""

#[Technical Assistance | Tone: Neutral | Urgency: Medium]
email3 = """

Subject: Installation help for Aero-Max security camera

Hi Support,

I’m Lin Wei (林伟), order #AM-554-993. I followed the quick-start guide, but the camera won’t connect to my 5 GHz Wi-Fi. Router is a TP-Link AX50.

Could you call me at +86 139-2211-0444 or reply to linwei.cam@outlook.cn with advanced setup steps?

Regards,  
Wei  
Address: 23-2 Guangfu Rd, Huangpu, Guangzhou 510623, China
"""

#[Shipping & Delivery Updates | Tone: Unhappy | Urgency: Medium]
email4 = """Subject: Where is package #TRK-998776543DE? Leaving town soon!

Dear team,

I’m leaving on June 15 and my parcel still shows “Label created.”  
Recipient: Dr. Margot Klein, Gartenweg 8, 55116 Mainz, Germany, phone +49 151 6888 9900.  
Please confirm the revised ETA or upgrade to express—happy to pay the difference.

Feeling a bit stressed; would appreciate quick clarification.

Kind regards,  
Margot
"""

#Testing Data

email5 = """Subject: Return request – wrong color delivered (Order #EU-445821)

Hello Returns Team,

I’m Pierre Lefèvre, order arrived today at 27 Rue des Lilas, 33000 Bordeaux, France but you sent the *Crimson Red* headphones instead of *Classic Black*.

Please advise on getting a prepaid label or arranging a swap.  
Email: p.lefevre@free.fr, phone +33 6 78 55 22 11.

Merci beaucoup,  
Pierre
"""

email6 = """Subject: DEFECTIVE air fryer exploded oil – urgent replacement demanded

To whom it may concern,

This is Sophia Hernandez (Order #MX-24681012). Last night the “Quick-Crisp 6 L” air fryer sprayed hot oil across my kitchen, nearly burning my son!  
Serial No: QC6-A1B2-C3D4.

I need an *immediate* replacement or full refund plus compensation for cleaning costs (receipt attached). Reach me at sophia.hdz@email.com or +52 55 2312 7766.  
Address: Calle Lago Garda 15, 1-A, 11520 CDMX, Mexico.

I’m furious—please act fast.

— Sophia
"""

email7 = """Subject: Double charge on credit card - invoice INV-2025-5566

Hi Billing,

I’m Khalid Al-Mazrouei, customer #C102233. My Mastercard (•••• •••• •••• 8120) was charged AED 1,299 twice on 5 June 2025.  
Please reverse the extra transaction.

Statement attached. Contact: +971 50 123 4567, khalid.am@protonmail.com.  
Postal: Villa 42, Al Raha Gardens, Abu Dhabi, UAE.

Regards,  
Khalid
"""

email8 = """Subject: Cannot reset password – account locked after 3 attempts

Hello,

I’m Emma-Jane O’Neill, username ejoneill88. The reset e-mail never arrives (checked spam).  
Could you manually reset or update the address to emma-jane.oneill@icloud.com?

Phone: +353 87 665 9020  
Current address on file: 74 Sandymount Rd, Dublin 4, Ireland.

Thanks,  
Emma-Jane
"""

email9 = """Subject: Missing loyalty points for last week’s purchase 🤔

Hey Rewards Crew!

Love the store ☺️ but my DragonFly Points balance didn’t update after Order #UK-PO-98456 (£289.99 on 4 June).  
Member ID DF-448822. Could you add the points and confirm?  

Reach me: luke.bennett@gmail.com, mobile +44 7900 123 456.  
Shipping address: Flat 7, 10 Jubilee Pl, London SW3 3TD, UK.

Cheers,  
Luke
"""

email10 = """Subject: Feedback on recent chatbot experience – could be better

Hi Customer Care,

I’m Dr. Aisha Yusuf, e-mail a.yusuf@postgrad.edu.ng, phone +234 803 555 0190.  
After yesterday’s chat session (Case ID CC-4321), I felt the bot looped the same FAQs and wasted 20 minutes.

Overall I like your products, but human escalation should happen sooner.  
Happy to discuss; my address: 12 Admiralty Way, Lekki Phase 1, Lagos, Nigeria.

Thank you for listening!

Best regards,  
Aisha
"""

# All in one list if you prefer iterable access:
sample_emails = [
    email1, email2, email3, email4, email5,
    email6, email7, email8, email9, email10,
]

In [ ]:
!pip install langsmith

In [ ]:
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_API_KEY"] = "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"
os.environ["LANGSMITH_PROJECT"] = "Project-Southafrica"

## PII (Redaction)

In [ ]:
import re, spacy
from typing import List

REGEX_PATTERNS: List[tuple[str, str]] = [
    (r"\b[\w.-]+@[\w.-]+\.\w+\b", "[REDACTED_EMAIL]"),
    (r"\b(?:\+?\d{1,3})?[-.\s]?\(?\d{2,4}\)?[-.\s]?\d{3,4}[-.\s]?\d{3,4}\b", "[REDACTED_PHONE]"),
    (r"\b\d{4}[-\s]?\d{4}[-\s]?\d{4}[-\s]?\d{4}\b", "[REDACTED_CARD]"),
    (r"\bORD-\d{6,8}\b", "[REDACTED_ORDER]"),
]

nlp = spacy.load("en_core_web_md", disable=["tagger", "parser", "lemmatizer"])

def redact(text: str) -> str:
    for pattern, repl in REGEX_PATTERNS:
        text = re.sub(pattern, repl, text, flags=re.I)
    doc = nlp(text)
    for ent in reversed(doc.ents):
        if ent.label_ in {"LOC", "PERSON", "GPE", "DATE"}:
            text = text[:ent.start_char] + f"[REDACTED_{ent.label_}]" + text[ent.end_char:]
    return text

AttributeError: module 'numpy' has no attribute '_no_nep50_warning'

In [ ]:
for email in sample_emails:
    print(redact(email))

# Classification

In [ ]:
classification_prompt = """
You are an email classification system for a customer support team. 
Categorize the following email into ONE of these 10 support groups:

1. **Product Consultation (Presales)**: Questions about product features, compatibility, or usage before purchase.
2. **Order Support (Presales)**: Issues with ordering, payment options, or shipping selection.
3. **Technical Assistance (Presales/Aftersales)**: Help with installation, setup, or troubleshooting.
4. **Shipping and Delivery Updates (Aftersales)**: Status, tracking, or delivery date inquiries.
5. **Returns and Exchanges Management (Aftersales)**: Returns, exchanges, or refund requests.
6. **Claims and Product Defects (Aftersales)**: Reports of defective/damaged items.
7. **Payment and Billing Support (Aftersales)**: Invoices, payment issues, or credit problems.
8. **Customer Account Support (Aftersales)**: Password resets, account changes.
9. **Loyalty Programs and Discounts (Presales/Aftersales)**: Vouchers, discounts, or loyalty programs.
10. **Customer Feedback and Complaints (Aftersales)**: Complaints or feedback submissions.

**Rules:**
- Respond ONLY with the exact category name (e.g., "Shipping and Delivery Updates (Aftersales)").
- If unsure, pick the most likely option.

**Email to classify:**
"{email_text}"

**Classification:**
"""
@traceable
async def classify_email(email_text):

    client = genai.Client(
        http_options=HttpOptions(api_version="v1"), 
        vertexai=True,
        project="ogcs-av8t-ailaboratory",
        location="europe-west1"
    )

    response = await client.aio.models.generate_content(
        model="gemini-2.0-flash-001",
        contents=email_text,
        config=types.GenerateContentConfig(
            system_instruction=[classification_prompt]
        ),
    )

    return response.text.rstrip("\n")

# Sentiment Analysis

In [ ]:
@traceable
def analyze_sentiment(text: str) -> dict:
    """
    Analyzes the sentiment of a given text using the Google Cloud Natural Language API
    and returns a detailed dictionary containing sentiment scores, magnitude,
    language, and sentiment breakdown by sentence.

    This function utilizes the `language_v2.LanguageServiceClient` to process the
    input text and determine its overall sentiment, as well as the sentiment of
    individual sentences within the text.

    The sentiment is categorized into "Very unhappy", "Unhappy", "Neutral",
    "Happy", and "Very Happy" based on predefined score ranges.

    Args:
        text (str): The input text (e.g., content of an email or message) to be
                    analyzed for sentiment.

    Returns:
        dict: A dictionary containing the sentiment analysis results.
              The structure of the dictionary is as follows:
              {
                  "sentiment_category": str,  # Overall sentiment ("Very unhappy", "Happy", etc.)
                  "score": float,             # Overall sentiment score (-1.0 to 1.0)
                  "magnitude": float,         # Overall sentiment magnitude (0.0 to +inf)
                  "language": str,            # Detected language of the text (e.g., "en")
                  "sentences": list           # List of dictionaries, each representing a sentence:
                      [
                          {
                              "text": str,          # The content of the sentence
                              "score": float,       # Sentiment score for the sentence
                              "magnitude": float,   # Sentiment magnitude for the sentence
                              "sentiment_label": str # Categorized sentiment for the sentence
                          },
                          ...
                      ]
              }
    """

    client = language_v2.LanguageServiceClient()

    document = {
        "content": text,
        "type_": language_v2.Document.Type.PLAIN_TEXT,
    }

    encoding_type = language_v2.EncodingType.UTF8

    response = client.analyze_sentiment(
        request={"document": document, "encoding_type": encoding_type}
    )

    score = response.document_sentiment.score
    magnitude = response.document_sentiment.magnitude

    def interpret_score(score: float) -> str:
        if score <= -0.35:
            return "Very unhappy"
        elif score <= -0.1:
            return "Unhappy"
        elif score < 0.25:
            return "Neutral"
        elif score < 0.75:
            return "Happy"
        else:
            return "Very Happy"

    result = {
        "sentiment_category": interpret_score(score),
        "score": score,
        "magnitude": magnitude,
        "language": response.language_code,
        "sentences": []
    }

    for sentence in response.sentences:
        sentence_score = sentence.sentiment.score
        result["sentences"].append({
            "text": sentence.text.content,
            "score": sentence_score,
            "magnitude": sentence.sentiment.magnitude,
            "sentiment_label": interpret_score(sentence_score)
        })

    return result


# Urgency

In [ ]:
@traceable
def define_urgency(category: str, sentiment: str) -> str:
    """
    Calculates an urgency score based on the category of the customer interaction
    and the sentiment expressed by the customer.

    The urgency score is determined by:
    1.  **Category:** Different categories are assigned varying urgency points.
        -   Aftersales categories (Shipping, Returns, Claims, Payment) add 3 points.
        -   Presales/Aftersales categories (Product Consultation, Order Support,
            Technical Assistance, Customer Account) add 2 points.
        -   Less urgent categories (Loyalty Programs, Customer Feedback) add 1 point.
    2.  **Sentiment:** Customer sentiment further modifies the urgency.
        -   "Very unhappy" sentiment adds 2 points.
        -   "Unhappy" sentiment adds 1 point.

    The final urgency score is the sum of points from both category and sentiment.

    Args:
        category (str): The classification of the customer interaction (e.g.,
                        "Shipping and Delivery Updates (Aftersales)",
                        "Product Consultation (Presales)").
        sentiment (str): The sentiment expressed by the customer (e.g.,
                         "Very unhappy", "Unhappy", "Neutral", "Happy").

    Returns:
        int: An integer representing the calculated urgency score.
             A higher score indicates greater urgency.
    """

    urgency_score = 0

    if category in ["Shipping and Delivery Updates (Aftersales)", 
                    "Returns and Exchanges Management (Aftersales)",
                    "Claims and Product Defects (Aftersales)",
                   "Payment and Billing Support (Aftersales)"]:
        urgency_score += 3
    elif category in ["Product Consultation (Presales)", 
                      "Order Support (Presales)",
                     "Technical Assistance (Presales/Aftersales)",
                      "Customer Account Support (Aftersales)"]:
        urgency_score += 2
    elif category in ["Loyalty Programs and Discounts (Presales/Aftersales)", 
                      "Customer Feedback and Complaints (Aftersales)"]:
        urgency_score += 1

    if sentiment == "Very unhappy":
        urgency_score += 2
    elif sentiment == "Unhappy":
        urgency_score += 1

    return urgency_score

# Draft Reply

In [ ]:
from google import genai
from google.genai import types
from google.genai.types import HttpOptions

@traceable
async def create_draft_reply (email_text: str) -> str:
    client = genai.Client(
        http_options=HttpOptions(api_version="v1"), 
        vertexai=True,
        project="ogcs-av8t-ailaboratory",
        location="europe-west1"
    )
    response = await client.aio.models.generate_content(
        model='gemini-2.0-flash-001',
        contents= email1,
        config=types.GenerateContentConfig(
            system_instruction=
              [
                'You are generating a brand-aligned draft reply template from the Otto Group in response to a customer message.'
                'Do not include any personal information about the customer in the template.'
              ]
        ),
    )
    return response.text

# Put all together

In [ ]:
for email in sample_emails:

    email_text = email

    support_team = await classify_email(email_text) 
    sentiment = analyze_sentiment(email_text)
    urgency = define_urgency(support_team, str(sentiment["sentiment_category"]))
    draft_reply = await create_draft_reply(email_text)

    print(email_text)
    print("Support Team:", support_team)
    print("Sentiment Category:", sentiment["sentiment_category"])
    print("Sentiment Score:", sentiment["score"])
    print("Urgency:", urgency)
    print("Draft Reply:", draft_reply)

# Store into JSON

In [ ]:
    upload_data("unhappy-user@gmail.com",
                "Unhappy User",
                email_text,
                "questions",
                support_team,
                sentiment["sentiment_category"],
                sentiment["score"],
                sentiment["magnitude"],
                urgency,
                draft_reply)     

! pip install google-genai

## Classification

https://cloud.google.com/natural-language/docs/reference/rpc/google.cloud.language.v2#classificationcategory

In [ ]:
!pip install mysql.connector

In [ ]:
import mysql.connector

def create_database_connection():
    hostname = "e7b80.h.filess.io"
    database = "Ottomatic_successin"
    port = "3307"
    username = "Ottomatic_successin"
    password = "5d02edcb6c8c37a88920af94aef69e90a046f183"

    connection = mysql.connector.connect(
        host=hostname,
        database=database,
        user=username,
        password=password,
        port=port
    )

    return connection

In [ ]:
def upload_data(email_adress:str,
               name: str,
               original_email: str,
               questions: str,
               support_team: str,
               sentiment_category: str,
               sentiment_score: float,
               sentiment_magnitude: float,
               urgency_category: str,
               reply: str):
    connection = create_database_connection()

    if connection.is_connected():
        cursor = connection.cursor()
    
        insert_query = "INSERT INTO mails (email_adress, name, original_email, questions, support_team, sentiment_category, sentiment_score, sentiment_magnitude, urgency_category, reply) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        cursor.execute(insert_query, (email_adress, name, original_email, questions, support_team, sentiment_category, sentiment_score, sentiment_magnitude, urgency_category, reply))
        connection.commit()

    if connection and connection.is_connected():
        cursor.close()
        connection.close()

    return True

In [ ]:
from google.cloud import storage
import os

#downloads the content located at the given src url to the provided path
#force_retrieve allows for a download, even if the content is already present
def download(bucket_name:str, blob_path:str, local_path:str, force_retrieve:bool=False) -> int:
    """Downloads a file from a Google Cloud Storage bucket to the local file system.

    Args:
        bucket_name (str): The name of the bucket from which the file will be downloaded.
        blob_path (str): The path within the bucket of the file to be downloaded.
        local_path (str): The path to the file on the local file system.
        force_retrieve (bool, optional): If set to True, the file will be downloaded even if it already exists locally. Default is False.

    Returns: 0 if file was found locally, 1 if file was successfully downloaded
    
    Return type: int

    Raises:
        google.cloud.exceptions.NotFound: If the specified bucket or blob does not exist.
        google.cloud.exceptions.Forbidden: If the authenticated user does not have read access to the bucket.
        google.api_core.exceptions.GoogleAPICallError: If there was an error during the API call
    """ 
    #check if file exists locally or force flag is set
    if (not(os.path.exists(local_path)) or force_retrieve):
        print(f"Requested file {local_path} not found locally or force_retrieve flag set. Downloading file from GCS.")
        storage_client = storage.Client()
        bucket = storage_client.bucket(bucket_name)
        
        blob = bucket.blob(blob_path)
        blob.download_to_filename(local_path)

        print(f"Downloaded storage object {blob_path} from {bucket_name} to local file {local_path}")
    else:
        print(f"Requested file {local_path} found locally.")

def store_result(bucket_name:str, blob_path:str, local_path:str) -> None:
    """Uploads a file from the local file system to a Google Cloud Storage bucket.

    Args:
        bucket_name (str): The name of the bucket to which the file will be uploaded.
        blob_path (str): The path within the bucket where the file will be stored.
        local_path (str): The path to the file on the local file system.

    Returns:
        None

    Raises:
        google.cloud.exceptions.NotFound: If the specified bucket does not exist.
        google.cloud.exceptions.Forbidden: If the authenticated user does not have write access to the bucket.
        google.api_core.exceptions.GoogleAPICallError: If there was an error during the API call.
    """
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(blob_path)

    storage.blob._DEFAULT_CHUNKSIZE = 5 * 1024* 1024  # should work if you can _always_ upload at least 5 MB per minute
    print('Trying to upload a file')
    blob.upload_from_filename(local_path)

    print(f"Successfully uploaded file {local_path} to gcp bucket {bucket_name} under path {blob_path}")

store_result("hackathon-team2-bucket","test.txt","test.txt")
download("hackathon-team2-bucket","test.txt","test.txt",force_retrieve=True)